### LCEL:Langchain表达式语言
- 一种在Langchain之上封装的高级解释语言
- 简化链条开发，支持真实生产环境而发明
---
更好的流式支持 更好的异步支持 优化执行时间 支持重试和反馈 轻松获取中间步骤 输入输出强验证 无缝追踪集成 无缝部署集成

In [1]:
# 一个简单的lcel示例

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi

prompt=ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")

llm=ChatTongyi(model="qwen-plus",temperature=0.5,max_token=1000)

output_parser=StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"topic":"流氓"})

'有一天，一个自称“街头霸王”的小流氓在公园长椅上睡觉。他特意把书包放在肚子上，好像在守护什么宝贝一样。\n\n这时，一位老太太走过来问：“孩子啊，你睡得这么香，是不是把钱包放肚子里了？”\n\n小流氓一听，立刻坐起来说：“奶奶，您真逗！我这是防着那些坏人偷我书包。”\n\n老太太笑了笑，指了指他的背后，说：“那你背后的那个牌子是谁挂的呢？”\n\n小流氓扭头一看，发现有人恶作剧，在他背后挂了个牌子，上面写着：“我是菜鸟，请多关照。” \n\n从此，“街头霸王”再也不敢轻易吹牛了！\n\n这个故事告诉我们：做人要低调，不然容易被“打脸”。 😄'

#### Prompt

In [2]:
prompt_value=prompt.invoke({"topic":"流氓"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='给我讲一个关于流氓的笑话', additional_kwargs={}, response_metadata={})])

In [3]:
prompt_value.to_messages()

[HumanMessage(content='给我讲一个关于流氓的笑话', additional_kwargs={}, response_metadata={})]

In [4]:
prompt_value.to_string()

'Human: 给我讲一个关于流氓的笑话'

#### Model

In [5]:
message=llm.invoke(prompt_value)
message

AIMessage(content='有一天，一个自称“街头霸王”的小流氓在公园长椅上睡着了。他做了一个梦，梦见自己变成了超人，在城市上空飞翔，拯救世界。\n\n突然，一只鸽子飞过来拉了一坨屎，正好掉在他的头上。他气得大喊：“谁干的？出来！”\n\n这时，树上的另一只鸽子淡定地回答：“不好意思啊兄弟，你别生气，这事儿咱能坐下来聊吗？” \n\n小流氓醒来后特别尴尬，因为周围的人都在看着他，而他的脸上真有一坨鸽子屎呢！\n\n这个笑话告诉我们：即使是自以为威风凛凛的人，也难免有被生活“戏弄”的时候。保持幽默感和谦逊很重要哦！', additional_kwargs={}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': '4463cc3d-40e1-97f5-bc22-b7c5bf3561ce', 'token_usage': {'input_tokens': 15, 'output_tokens': 156, 'total_tokens': 171, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--cb2d5dba-cfad-4524-993f-e0a8ccdbd59d-0')

#### Output Parser

In [6]:
output_parser.invoke(message)

'有一天，一个自称“街头霸王”的小流氓在公园长椅上睡着了。他做了一个梦，梦见自己变成了超人，在城市上空飞翔，拯救世界。\n\n突然，一只鸽子飞过来拉了一坨屎，正好掉在他的头上。他气得大喊：“谁干的？出来！”\n\n这时，树上的另一只鸽子淡定地回答：“不好意思啊兄弟，你别生气，这事儿咱能坐下来聊吗？” \n\n小流氓醒来后特别尴尬，因为周围的人都在看着他，而他的脸上真有一坨鸽子屎呢！\n\n这个笑话告诉我们：即使是自以为威风凛凛的人，也难免有被生活“戏弄”的时候。保持幽默感和谦逊很重要哦！'

#### LCEL的Pipeline

In [7]:
''' Input --dict--> PromptTemplate --PromptValue--> ChatModel --ChatMessage--> StrOutputParser --String--> Result '''

' Input --dict--> PromptTemplate --PromptValue--> ChatModel --ChatMessage--> StrOutputParser --String--> Result '

#### RAG Search Example
- 建立向量数据
- 使用RAG增强

In [8]:
from operator import itemgetter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.embeddings import DashScopeEmbeddings

vectorstore=FAISS.from_texts(
    ["我是你爸爸真伟大，养你这么大"],
    embedding=DashScopeEmbeddings()
)

retriever=vectorstore.as_retriever()

template='''根据上下文回答问题：{context}\n问题：{question}'''

prompt=ChatPromptTemplate.from_template(template)

llm=ChatTongyi(model="qwen-plus",temperature=0.2,max_token=1000)

chain=(
    {"context":retriever,"question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
chain.invoke("我是谁？")

'根据提供的文档内容“我是你爸爸真伟大，养你这么大”，可以推断出问题是关于亲子关系的。因此，答案是：\n\n**你是对方的爸爸。**'

In [10]:
# 自定义也非常简单

template="""Answer the question based onlu on the following context:
{context}
Question:{question}
Answer in the following language:{language}
"""

prompt=ChatPromptTemplate.from_template(template)

chain=(
    {
        "context":itemgetter("question") | retriever,
        "question":itemgetter("question"),
        "language":itemgetter("language")
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
chain.invoke(
    {
        "question":"我是谁？",
        "language":"English"
    }
)

'I am your great father who has raised you.'